# Kayıp Fonksiyonları

Bu alıştırmada, Kayıp fonksiyonlarının `LinearRegression` modeli üzerindeki etkilerini karşılaştıracaksınız.

👇 Bu zorluk için kullanmak üzere bir CSV dosyası indirelim ve onu bir DataFrame'e dönüştürelim

In [1]:
import pandas as pd

data = pd.read_csv("https://d32aokrjazspmn.cloudfront.net/materials/loss_functions_dataset.csv")
data.sample(5)

,Relative Compactness,Surface Area,Wall Area,Roof Area,Overall Height,Glazing Area,Average Temperature
754,0.69,735.0,294.0,220.5,3.5,0.40,15.455
478,0.62,808.5,367.5,220.5,3.5,0.25,14.840
365,0.71,710.5,269.5,220.5,3.5,0.25,13.845
451,0.79,637.0,343.0,147.0,7.0,0.25,38.690
763,0.64,784.0,343.0,220.5,3.5,0.40,19.640


🎯 Göreviniz, tasarımına göre bir seranın içindeki ortalama sıcaklığı tahmin etmektir. Sıcaklık tahminleriniz, her bir bitki için iklim ihtiyaçlarına göre uygun sera tasarımını seçmenize yardımcı olacaktır.

🌿 Bitkilerin küçük sıcaklık değişimlerini kaldırabildiğini, ancak sıcaklık değişimleri arttıkça katlanarak daha duyarlı hale geldiğini biliyorsunuz.

## 1. Teori

❓ Teorik olarak, bitkileri öldürme riskini sınırlamak için modelinizi hangi Kayıp fonksiyonu üzerinde eğitirsiniz?

<details>
<summary> 🆘 Cevap </summary>
    
Teorik olarak, Ortalama Kare Hata (MSE) Kayıp fonksiyonunu kullanırsınız. Bu, aykırı tahminleri cezalandırır ve modelinizin büyük hatalar yapmasını engeller. Bu, daha küçük sıcaklık değişimleri ve bitkiler için daha düşük risk sağlayacaktır.

</details>

> MSE (L2 / squared error)

## 2. Uygulama

### 2.1 Ön İşleme

❓ Özellikleri standartlaştırın

In [2]:
data.columns

Index(['Relative Compactness', 'Surface Area', 'Wall Area', 'Roof Area',
       'Overall Height', 'Glazing Area', 'Average Temperature'],
      dtype='object')

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Features and target
X = data.drop(columns='Average Temperature')
y = data['Average Temperature']

# split (before preprocess)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size= 0.2, random_state= 42
)

# scale to Train
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create a df for protect to column names
X_train_scaled = pd.DataFrame(X_train_scaled, columns= X.columns , index= X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns= X.columns, index= X_test.index)

X_train_scaled.head()

,Relative Compactness,Surface Area,Wall Area,Roof Area,Overall Height,Glazing Area
60,0.553671,-0.696222,-0.007372,-0.679048,1.016421,-1.010300
618,-1.155118,1.250664,0.558439,0.957063,-0.983844,1.227790
346,0.933402,-0.974349,-0.573184,-0.679048,1.016421,0.108745
294,1.313133,-1.252476,-0.007372,-1.224418,1.016421,0.108745
231,-0.965252,0.972537,-0.007372,0.957063,-0.983844,-1.010300


### 2.2 Modelleme

Bu bölümde, farklı Kayıp fonksiyonları üzerinde optimize edilmiş modelleri değerlendirerek teoriyi doğrulayacaksınız.

### En Küçük Kareler (MSE) Kaybı

❓ **En Küçük Kareler Kaybı** (MSE) üzerinde **Stokastik Gradyan İnişi** (SGD) ile optimize edilmiş bir Doğrusal Regresyon modelini **10-Katlı Çapraz doğrula**

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score , cross_val_predict
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import max_error


pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("model", SGDRegressor(loss="squared_error", random_state=42))
])

scores = cross_val_score(
    pipe,
    X_train,
    y_train,
    cv=10,
    scoring="r2"
)
r2 = scores.mean()

y_pred = cross_val_predict(
    pipe,
    X_train,
    y_train,
    cv=10
)

max_error_celsius = max_error(y_train,y_pred)

print("r2: ", r2)


r2:  0.9079703032619515


❓ Hesaplayın:
- Ortalama çapraz doğrulanmış R2 skoru ve bunu `r2` değişkeninde kaydedin
- Tüm katlarınızın °C cinsinden en büyük tek tahmin hatasını hesaplayın ve `max_error_celsius` değişkeninde kaydedin

(İpucu: `max_error` sklearn'de kabul edilen bir puanlama metriğidir)

In [14]:
print("max_error_celcius: ", max_error_celsius)

max_error_celcius:  9.339185010473685


### Ortalama Mutlak Hata (MAE) Kaybı

Peki modelimizi MAE üzerinde optimize edersek ne olur?

❓ **MAE** Kaybı üzerinde **Stokastik Gradyan İnişi** (SGD) ile optimize edilmiş bir Doğrusal Regresyon modelini **10-Katlı Çapraz doğrula**

<details>
<summary>💡 İpuçları</summary>

- MAE kaybı `SGDRegressor`'da doğrudan belirtilemez. Doğru parametreleri ayarlayarak tasarlanması gerekir

</details>

In [7]:
pipe_mae = Pipeline([
    ("scaler", StandardScaler()),
    ("model", SGDRegressor(loss="epsilon_insensitive", random_state=42))
])

scores_mae = cross_val_score(
    pipe_mae,
    X_train,
    y_train,
    cv=10,
    scoring="r2"
)

r2_mae = scores_mae.mean()

y_pred_mae = cross_val_predict(
    pipe_mae,
    X_train,
    y_train,
    cv=10
)

max_error_mae = max_error(y_train, y_pred_mae)

print("r2_mae:", r2_mae)


r2_mae: 0.8806330923832263


❓ Hesaplayın:
- Ortalama çapraz doğrulanmış R2 skoru, bunu `r2_mae`'de saklayın
- Tüm katlarınızın en büyük tek tahmin hatasını, bunu `max_error_mae`'de saklayın

In [8]:
print("max_error_mae:", max_error_mae)

max_error_mae: 11.87773614393172


## 3. Sonuç

❓ Değerlendirdiğiniz modellerden hangisi göreviniz için en uygun görünüyor?

<details>
<summary> 🆘Cevap </summary>
    
İki model arasında ortalama çapraz doğrulanmış r2 skorları yaklaşık olarak benzer olmasına rağmen, MAE üzerinde optimize edilen modelin zaman zaman daha büyük hatalar yapma şansı daha fazladır, bu da bitkileri öldürme riskini artırır!
    
</details>

> r2

# 🏁 Kodunuzu kontrol edin ve notebook'unuzu gönderin

In [15]:
from nbresult import ChallengeResult

result = ChallengeResult(
    'loss_functions',
    r2 = r2,
    r2_mae = r2_mae,
    max_error = max_error_celsius,
    max_error_mae = max_error_mae
)

result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /home/gunay/.pyenv/versions/workintech/bin/python
cachedir: .pytest_cache
rootdir: /home/gunay/code/01-Python/sprint-16/01-machine-learning/S16D4-S-loss-functions/tests
plugins: anyio-4.8.0, typeguard-4.4.2
collecting ... collected 3 items

test_loss_functions.py::TestLossFunctions::test_max_error_order PASSED   [ 33%]
test_loss_functions.py::TestLossFunctions::test_r2 PASSED                [ 66%]
test_loss_functions.py::TestLossFunctions::test_r2_mae PASSED            [100%]

============================== 3 passed in 0.27s ===============================


💯 You can commit your code:

git add tests/loss_functions.pickle

git commit -m 'Completed loss_functions step'

git push origin master

